<a href="https://colab.research.google.com/github/fid0did0/WebScraper/blob/main/borse_web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.Bx251CGqNw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.YxQDelXjBD/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.U1H4qypeIQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update > /dev/null
!apt-get install -y chromium chromium-chromedriver > /dev/null
!pip install selenium > /dev/null
!pip install chromedriver-autoinstaller > /dev/null

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
chromium is already the newest version (90.0.4430.212-1~deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [ ]:
import sys
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
#sys.path.insert(0, '/usr/bin/chromedriver')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')  # Required in Colab environment

# Launch Chrome browser in headless mode
driver = webdriver.Chrome(options=chrome_options)


WebDriverException: ignored

In [ ]:
def got_next_pg(driver):
  next_pg_button_xpath="/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-page-bar[1]/div/div/button[6]"
  element_to_chck = driver.find_element(By.XPATH, next_pg_button_xpath)
  str_to_chck = element_to_chck.get_attribute("title")
  if str_to_chck=="Zeige Seite 2":
    return False
  else:
    return True

In [ ]:

driver.get("https://www.boerse-frankfurt.de/suchergebnisse/Frankreich")

anleihen_button_xpath="/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-data-menue/div/div/div/drag-scroll/div/div/button[1]"
next_pg_button_xpath="/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-page-bar[1]/div/div/button[6]"

WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, anleihen_button_xpath)))

anleihen_button = driver.find_element(By.XPATH, anleihen_button_xpath)
stich_button = driver.find_element(By.XPATH, '/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/app-data-menue/div/div/div/drag-scroll/div/div/button[2]')

anleihen_button.click()
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, next_pg_button_xpath)))

entry_link_list = []
for page_idx in range(2):
  #print("start processing page %d" % page_idx)
  next_pg_button=driver.find_element(By.XPATH, next_pg_button_xpath)
  for k in range(1,30):
    entry_xpath='/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/div[2]/table/tbody/tr[%d]/td[1]/a' % (k)
    entry = driver.find_element(By.XPATH, entry_xpath)
    #print(entry.get_attribute("title"))
    entry_link_list.append(entry.get_attribute("href"))

  first_row_name=driver.find_element(By.XPATH, first_row_xpath).get_attribute("title")
  if next_pg_button.is_enabled:
    print("page %d downloaded" % page_idx)
    next_pg_button.click()
    WebDriverWait(driver, 3).until(got_next_pg)
  else:
    print("last page %d downloaded" % page_idx)


#entry_xpath='/html/body/app-root/app-wrapper/div/div[2]/app-global-search-result-overview/div/div[2]/table/tbody/tr[1]/td[1]/a'
#entry = driver.find_element(By.XPATH, entry_xpath)
#print(entry.get_attribute("title"))


In [ ]:
for enty_link_iter in entry_link_list:
  #print(enty_link_iter)
  driver.get(enty_link_iter)
  anleihe_name_xpath = "/html/body/app-root/app-wrapper/div/div[2]/app-bond/div[1]/div/app-widget-datasheet-header/div/div/div/div/div[1]/div/h1"
  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, anleihe_name_xpath)))

  anleihe_name = driver.find_element(By.XPATH, anleihe_name_xpath)
  #print(anleihe_name.text)
  curs_tab = driver.find_element(By.TAG_NAME, "app-widget-price-box")
  last_price = curs_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
  #print(last_price.text)

  rendite_tab = driver.find_element(By.TAG_NAME, "app-widget-key-figures")
  try:
    rendite = rendite_tab.find_element(By.XPATH, "div/div/table/tbody/tr[1]/td[2]")
    rendite_txt=rendite.text
  except NoSuchElementException:
    rendite_txt="N/A"

  print("%-40sprice: %-14srendite: %s" % (anleihe_name.text, last_price.text, rendite_txt))


In [ ]:
driver.quit()